In [16]:
# Function to calculate Awesome Oscillator
def calculate_ao(data, short_window=5, long_window=34, price_col='Close'):
    if price_col not in data.columns:
        raise KeyError(f"Column '{price_col}' not found in DataFrame")
    data['SMA_short'] = data[price_col].rolling(window=short_window).mean()
    data['SMA_long'] = data[price_col].rolling(window=long_window).mean()
    data['AO'] = data['SMA_short'] - data['SMA_long']
    return data

# Function to calculate RSI
def calculate_rsi(data, periods=14, price_col='Close'):
    if price_col not in data.columns:
        raise KeyError(f"Column '{price_col}' not found in DataFrame")
    delta = data[price_col].diff()
    up = delta.clip(lower=0)
    down = (-delta).clip(lower=0)
    rs = up.ewm(com=periods-1, adjust=False).mean() / down.ewm(com=periods-1, adjust=False).mean()
    data['RSI'] = 100 - 100 / (1 + rs)
    return data


In [17]:
import yfinance as yf
import pandas as pd


# Define the tickers for Bitcoin and Ethereum
tickers = ['BTC-USD', 'ETH-USD']

# Define the date range
start_date = '2023-04-01'
end_date = '2025-04-01'

# Download daily data and calculate indicators
daily_data = {}
required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
for ticker in tickers:
    try:
        # Fetch data for one ticker at a time
        df = yf.download(ticker, start=start_date, end=end_date, interval='1d')
        if df.empty:
            raise ValueError(f"No data returned for {ticker}")
        
        # Handle multi-index columns
        if isinstance(df.columns, pd.MultiIndex):
            print(f"MultiIndex columns detected for {ticker}: {df.columns}")
            # Extract the first level of the multi-index (e.g., 'Close', 'High')
            df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
        
        # Standardize column names (case-insensitive)
        column_map = {col.lower(): col.capitalize() for col in df.columns}
        df = df.rename(columns=column_map)
        
        # Verify required columns
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise KeyError(f"Missing columns for {ticker}: {missing_cols}")
        
        print(f"Columns for {ticker} daily data: {list(df.columns)}")
        daily_data[ticker] = df
        
        # Calculate AO and RSI for daily data
        daily_data[ticker] = calculate_ao(daily_data[ticker])
        daily_data[ticker] = calculate_rsi(daily_data[ticker])
    except Exception as e:
        print(f"Error fetching daily data for {ticker}: {e}")
        continue

# Aggregate daily data into 10-day periods
ten_day_data = {}
for ticker in daily_data:
    try:
        df = daily_data[ticker].copy()
        print(f"Columns before aggregation for {ticker}: {list(df.columns)}")
        
        # Create a period index for 10-day intervals
        df['Period'] = (df.index - pd.Timestamp(start_date)).days // 10
        
        # Verify required columns before aggregation
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise KeyError(f"Missing columns for {ticker} before aggregation: {missing_cols}")
        
        # Group by period and aggregate
        aggregated = df.groupby('Period').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        })
        
        # Convert period back to dates (last day of each 10-day period)
        aggregated.index = pd.Timestamp(start_date) + pd.to_timedelta(aggregated.index * 10 + 9, unit='D')
        ten_day_data[ticker] = aggregated
        
        # Calculate AO and RSI for 10-day data as a new time frame
        ten_day_data[ticker] = calculate_ao(ten_day_data[ticker])
        ten_day_data[ticker] = calculate_rsi(ten_day_data[ticker])
        
        print(f"Columns after aggregation for {ticker}: {list(ten_day_data[ticker].columns)}")
    except Exception as e:
        print(f"Error aggregating 10-day data for {ticker}: {e}")
        continue

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

MultiIndex columns detected for BTC-USD: MultiIndex([( 'Close', 'BTC-USD'),
            (  'High', 'BTC-USD'),
            (   'Low', 'BTC-USD'),
            (  'Open', 'BTC-USD'),
            ('Volume', 'BTC-USD')],
           names=['Price', 'Ticker'])
Columns for BTC-USD daily data: ['Close', 'High', 'Low', 'Open', 'Volume']
MultiIndex columns detected for ETH-USD: MultiIndex([( 'Close', 'ETH-USD'),
            (  'High', 'ETH-USD'),
            (   'Low', 'ETH-USD'),
            (  'Open', 'ETH-USD'),
            ('Volume', 'ETH-USD')],
           names=['Price', 'Ticker'])
Columns for ETH-USD daily data: ['Close', 'High', 'Low', 'Open', 'Volume']
Columns before aggregation for BTC-USD: ['Close', 'High', 'Low', 'Open', 'Volume', 'SMA_short', 'SMA_long', 'AO', 'RSI']
Columns after aggregation for BTC-USD: ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_short', 'SMA_long', 'AO', 'RSI']
Columns before aggregation for ETH-USD: ['Close', 'High', 'Low', 'Open', 'Volume', 'SMA_short', 'SM

In [18]:
from IPython.display import display

# Display the data in Jupyter Notebook
for ticker in tickers:
    if ticker in daily_data:
        print(f"{ticker} Daily Data with AO and RSI:")
        display(daily_data[ticker][['Open', 'High', 'Low', 'Close', 'Volume', 'AO', 'RSI']].tail())
    else:
        print(f"No daily data available for {ticker}")
    if ticker in ten_day_data:
        print(f"\n{ticker} 10-Day Interval Data with AO and RSI:")
        display(ten_day_data[ticker][['Open', 'High', 'Low', 'Close', 'Volume', 'AO', 'RSI']].tail())
    else:
        print(f"No 10-day data available for {ticker}")

BTC-USD Daily Data with AO and RSI:


,Open,High,Low,Close,Volume,AO,RSI
Date,,,,,,,
2025-03-27,86896.257812,87786.726562,85837.937500,87177.101562,24413471941,786.727895,51.460344
2025-03-28,87185.234375,87489.859375,83557.640625,84353.148438,34198619509,806.029917,45.566208
2025-03-29,84352.070312,84567.335938,81634.140625,82597.585938,16969396135,228.009467,42.320969
2025-03-30,82596.984375,83505.000000,81573.250000,82334.523438,14763760943,-532.260340,41.840104
2025-03-31,82336.062500,83870.125000,81293.890625,82548.914062,29004228247,-1220.675919,42.414371



BTC-USD 10-Day Interval Data with AO and RSI:


,Open,High,Low,Close,Volume,AO,RSI
Period,,,,,,,
2025-02-28,95532.531250,99497.968750,78248.914062,84373.007812,482674144877,21865.222679,53.772268
2025-03-10,84373.867188,95043.437500,77420.593750,78532.000000,493117479148,17058.902091,48.650983
2025-03-20,78523.875000,87443.265625,76624.250000,84167.195312,304187645038,12609.685179,53.274577
2025-03-30,84164.539062,88758.726562,81573.250000,82334.523438,223126021475,9248.992624,51.645868
2025-04-09,82336.062500,83870.125000,81293.890625,82548.914062,29004228247,6102.117785,51.831389


ETH-USD Daily Data with AO and RSI:


,Open,High,Low,Close,Volume,AO,RSI
Date,,,,,,,
2025-03-27,2008.940552,2037.396118,1987.717773,2002.357422,11593274543,-103.276277,43.345227
2025-03-28,2002.410522,2015.454346,1863.010620,1895.502930,18160526498,-99.690636,37.960066
2025-03-29,1895.549683,1911.900879,1799.200806,1827.320312,12194771785,-120.487375,34.974196
2025-03-30,1827.311035,1847.570557,1769.412720,1806.218628,9854857162,-151.984639,34.080725
2025-03-31,1806.316528,1852.551392,1778.692261,1823.479980,15765030938,-169.417316,35.531575



ETH-USD 10-Day Interval Data with AO and RSI:


,Open,High,Low,Close,Volume,AO,RSI
Period,,,,,,,
2025-02-28,2669.213379,2850.610352,2076.169922,2237.905273,257539517652,-264.571668,37.794467
2025-03-10,2237.942139,2548.811768,1812.766846,1861.151367,241081283316,-490.931643,33.316957
2025-03-20,1859.777710,2068.764404,1760.941772,1982.099854,153537490770,-682.557300,35.940680
2025-03-30,1981.853027,2101.983398,1769.412720,1806.218628,114110026593,-808.552110,33.854632
2025-04-09,1806.316528,1852.551392,1778.692261,1823.479980,15765030938,-939.662228,34.257926
